In [32]:
!pip uninstall -y numpy torch
!pip uninstall -y sentence-transformers

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Found existing installation: sentence-transformers 2.7.0
Uninstalling sentence-transformers-2.7.0:
  Successfully uninstalled sentence-transformers-2.7.0


In [2]:
!pip install numpy==1.26.4
!pip install torch

  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
  Using cached torch-2.4.0-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
Using cached torch-2.4.0-cp310-cp310-manylinux1_x86_64.whl (797.2 MB)

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [9]:
!pip install sentence-transformers==2.7.0

  Using cached sentence_transformers-2.7.0-py3-none-any.whl.metadata (11 kB)
Using cached sentence_transformers-2.7.0-py3-none-any.whl (171 kB)

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [10]:
import pandas as pd
import numpy as np
print(np.__version__)

1.26.4


In [4]:
github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"

In [5]:
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [6]:
df = df.iloc[:300]

In [7]:
df

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp
...,...,...,...,...,...
295,An alternative way to load the data using the ...,Above users showed how to load the dataset dir...,8d209d6d,What is an alternative way to load the data us...,machine-learning-zoomcamp
296,You can directly download the dataset from Git...,Above users showed how to load the dataset dir...,8d209d6d,How can I directly download the dataset from G...,machine-learning-zoomcamp
297,You can fetch data for homework using the `req...,Above users showed how to load the dataset dir...,8d209d6d,Could you share a method to fetch data for hom...,machine-learning-zoomcamp
298,If the status code is 200 when downloading dat...,Above users showed how to load the dataset dir...,8d209d6d,What should I do if the status code is 200 whe...,machine-learning-zoomcamp


## Q1. Getting the embeddings model


In [12]:
model_name = "multi-qa-mpnet-base-dot-v1"

In [13]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
answer_llm = df.iloc[0].answer_llm

In [16]:
answer_llm_embedings = embedding_model.encode(answer_llm)
answer_llm_embedings[0]

-0.42244655


## Q2. Computing the dot product


In [19]:
from tqdm.notebook import tqdm


In [20]:
evaluations = list()
for item in tqdm(df.itertuples()):
    v = embedding_model.encode(item.answer_llm)
    w = embedding_model.encode(item.answer_orig)
    result = v.dot(w)
    evaluations.append(result)

0it [00:00, ?it/s]

In [22]:
percentil_75 = np.percentile(evaluations, 75)

In [23]:
percentil_75

31.67430877685547

## Q3. Computing the cosine

In [25]:
evaluations = list()
for item in tqdm(df.itertuples()):
    v = embedding_model.encode(item.answer_llm)
    w = embedding_model.encode(item.answer_orig)
    result = np.dot(v, w) / (np.linalg.norm(v) * np.linalg.norm(w))
    evaluations.append(result)
    

0it [00:00, ?it/s]

In [26]:
percentil_75 = np.percentile(evaluations, 75)

In [27]:
percentil_75

0.836234912276268

## Q4. Rouge

In [29]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [31]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df.iloc[10].to_dict()
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [32]:
scores['rouge-1']

{'r': 0.45454545454545453, 'p': 0.45454545454545453, 'f': 0.45454544954545456}

In [33]:
round(scores['rouge-1']['f'], 2)

0.45

## Q5. Average rouge score

In [36]:
avg_rouge = np.average([scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']]).round(2)
avg_rouge

0.35

## Q6. Average rouge score for all the data points

In [37]:
data_rouge = {
    'rouge_1': list(),
    'rouge_2': list(),
    'rouge_l': list()
}
for item in tqdm(df.itertuples()):
    scores = rouge_scorer.get_scores(item.answer_llm, item.answer_orig)[0]
    data_rouge['rouge_1'].append(scores['rouge-1']['f'])
    data_rouge['rouge_2'].append(scores['rouge-2']['f'])
    data_rouge['rouge_l'].append(scores['rouge-l']['f'])

0it [00:00, ?it/s]

In [38]:
df_rouge = pd.DataFrame(data_rouge)
df_rouge.head()

,rouge_1,rouge_2,rouge_l
0,0.095238,0.028169,0.095238
1,0.125000,0.055556,0.093750
2,0.415584,0.177778,0.389610
3,0.216216,0.047059,0.189189
4,0.142076,0.033898,0.120219


In [39]:
df_rouge.rouge_2.describe()

count    300.000000
mean       0.206965
std        0.153550
min        0.000000
25%        0.097809
50%        0.178671
75%        0.286181
max        0.739130
Name: rouge_2, dtype: float64


What's the average `rouge_2` across all the records?

- 0.10
- **0.20**
- 0.30
- 0.40